In [1]:
import torch
import torchvision
# import
import os
import pickle
import numpy as np
import PIL.Image

import scipy.io as sio
from datetime import datetime
import sys


sys.path.append('../model')
from C3D import C3D
                

sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel, save_video, normalise_vid, get_cnn_features, img_deprocess
from activation_maximization import generate_preferred

In [2]:
net = C3D.C3D()
param_file = os.path.join('../model','C3D', 'c3d.pickle')
net.load_state_dict(torch.load(param_file))
net.eval()

C3D(
  (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3a): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3b): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4b): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv5a): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=

In [3]:
net = torchvision.models.alexnet(pretrained=True)

In [4]:

#net = torchvision.models.vgg16(pretrained=True)

In [5]:
# image
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])

# preprocess 
norm = 255

In [17]:
#save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_simpleCNN_lower'#__file__.split('.')[0]
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [7]:
def hook(module, input, output):
    outputs.append(output.clone())

In [8]:
net.features[8]

Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [9]:
layer_list = ['fc8']
layer_list = ['conv4a']
layer = layer_list[0]

In [10]:
# initial image for the optimization

h, w = 224,224
initial_image = np.zeros(( h,w,3),dtype='float32')


initial_input = np.random.randint(0, 256, (h,w,3))

In [11]:
net

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [23]:

target_layer = "net.features[8]"
target_layer = "net.features[10]"

In [24]:
#targetlayer
exec_str_list = [target_layer +".register_forward_hook(hook)"]


exec("num_of_ch = "+target_layer+".weight.detach().numpy().shape[0]") #param_list[layer_value].shape[0]
num_of_img = 10
step = int(num_of_ch/num_of_img)
channel_list = range(0,num_of_ch,step)

ee = get_cnn_features(net,torch.Tensor(initial_input.transpose(2,0, 1)[np.newaxis]), exec_str_list)
feat_num = ee[0].detach().numpy().shape

In [25]:
# options
opts = {
    'img_mean': img_mean,
    'img_std': img_std,
    'exec_code': exec_str_list, # exection code inside the function

    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start': 1., # learning rate
    'lr_end': 1.,

    'momentum_start': 0.001, # gradient with momentum
    'momentum_end': 0.001,

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during
    'jitter_size': 4,
    
    'use_p_norm_reg': False,
    'p': 2,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    #'input_size': (16, 112,112,3),
    'input_size': (224,224,3),
    #'initial_input': None, # the initial image for the optimization (setting to None will use random noise as initial image)
    'initial_input': initial_input,
    }



In [26]:
channel_list = [14,56]

In [27]:
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # target units
    feat_size = feat_num
    y_index = int(feat_size[2]/2) # the unit in the center of feature map
    x_index = int(feat_size[3]/2) # the unit in the center of feature map
    feature_mask = np.zeros(feat_size)
    feature_mask[0,channel,y_index,x_index] = 1
    feature_mask[0,channel] = 1
    # weights for the target units
    feature_weight = np.zeros(feat_size, dtype=np.float32)
    feature_weight[:] = 1.
    #
    preferred_stim = generate_preferred(net, feature_mask, feature_weight=feature_weight, **opts)

    # save the results
    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.mat'
    sio.savemat(os.path.join(save_path,save_name),{'preferred_stim':preferred_stim})
    

    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.jpg'
    #save_video(normalise_vid(clip_extreme_pixel(preferred_stim,pct=0.04)), save_name, save_path )
    PIL.Image.fromarray(normalise_img(clip_extreme_pixel(preferred_stim, pct=0.04))).save(
                    os.path.join(save_path, save_name))


channel=14

iter=1; mean(abs(feat))=3.03844;
iter=2; mean(abs(feat))=4.96575;
iter=3; mean(abs(feat))=5.35829;
iter=4; mean(abs(feat))=4.67019;
iter=5; mean(abs(feat))=4.20545;
iter=6; mean(abs(feat))=0.514993;
iter=7; mean(abs(feat))=2.816;
iter=8; mean(abs(feat))=1.96075;
iter=9; mean(abs(feat))=1.87771;
iter=10; mean(abs(feat))=1.2028;
iter=11; mean(abs(feat))=0.378587;
iter=12; mean(abs(feat))=1.32078;
iter=13; mean(abs(feat))=1.72658;
iter=14; mean(abs(feat))=0.255907;
iter=15; mean(abs(feat))=1.06115;
iter=16; mean(abs(feat))=2.01658;
iter=17; mean(abs(feat))=0.120936;
iter=18; mean(abs(feat))=1.01427;
iter=19; mean(abs(feat))=1.35519;
iter=20; mean(abs(feat))=0.518563;
iter=21; mean(abs(feat))=1.52663;
iter=22; mean(abs(feat))=3.14852;
iter=23; mean(abs(feat))=0.801019;
iter=24; mean(abs(feat))=2.38674;
iter=25; mean(abs(feat))=4.80215;
iter=26; mean(abs(feat))=3.52805;
iter=27; mean(abs(feat))=2.28829;
iter=28; mean(abs(feat))=1.00747;
iter=29; mean(abs(feat))=5.697;
iter=30; 

iter=39; mean(abs(feat))=4.77915;
iter=40; mean(abs(feat))=4.20015;
iter=41; mean(abs(feat))=5.46153;
iter=42; mean(abs(feat))=5.83136;
iter=43; mean(abs(feat))=4.90981;
iter=44; mean(abs(feat))=7.27409;
iter=45; mean(abs(feat))=0.427473;
iter=46; mean(abs(feat))=5.28387;
iter=47; mean(abs(feat))=6.32106;
iter=48; mean(abs(feat))=6.17194;
iter=49; mean(abs(feat))=1.77491;
iter=50; mean(abs(feat))=8.54342;
iter=51; mean(abs(feat))=7.93211;
iter=52; mean(abs(feat))=7.272;
iter=53; mean(abs(feat))=8.37913;
iter=54; mean(abs(feat))=7.8945;
iter=55; mean(abs(feat))=8.54776;
iter=56; mean(abs(feat))=9.18525;
iter=57; mean(abs(feat))=7.02576;
iter=58; mean(abs(feat))=10.2983;
iter=59; mean(abs(feat))=6.74705;
iter=60; mean(abs(feat))=7.90552;
iter=61; mean(abs(feat))=6.83279;
iter=62; mean(abs(feat))=6.15096;
iter=63; mean(abs(feat))=9.34711;
iter=64; mean(abs(feat))=11.6663;
iter=65; mean(abs(feat))=8.69719;
iter=66; mean(abs(feat))=9.76905;
iter=67; mean(abs(feat))=12.4775;
iter=68; mean(ab

In [18]:
save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.jpg'
#save_video(normalise_vid(clip_extreme_pixel(preferred_stim,pct=0.04)), save_name, save_path )
PIL.Image.fromarray(normalise_img(clip_extreme_pixel(preferred_stim, pct=0.04))).save(
                os.path.join(save_path, save_name))

In [37]:
# video
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # target units
    feat_size = feat_num
    t_index = int(feat_size[2]/2)
    y_index = int(feat_size[3]/2) # the unit in the center of feature map
    x_index = int(feat_size[4]/2) # the unit in the center of feature map
    feature_mask = np.zeros(feat_size)
    feature_mask[0,channel,t_index, y_index,x_index] = 1

    # weights for the target units
    feature_weight = np.zeros(feat_size, dtype=np.float32)
    feature_weight[:] = 1.
    #
    preferred_vid = generate_preferred(net, feature_mask, feature_weight=feature_weight, **opts)

    # save the results
    save_name = 'preferred_img' + '_layer_' + str(layer) + '_channel_' + str(channel) + '.mat'
    sio.savemat(os.path.join(save_path,save_name),{'preferred_vid':preferred_vid})

    save_name = 'preferred_img' + '_layer_' + str(layer) + '_channel_' + str(channel) + '.avi'
    save_video(normalise_vid(clip_extreme_pixel(preferred_vid,pct=0.04)), save_name, save_path )


channel=23

1
iter=1; mean(abs(feat))=4.71797;
1
iter=2; mean(abs(feat))=328.048;
1
iter=3; mean(abs(feat))=627.787;
1
iter=4; mean(abs(feat))=822.265;
1
iter=5; mean(abs(feat))=1149.37;
1
iter=6; mean(abs(feat))=1239.92;
1
iter=7; mean(abs(feat))=1461.47;
1
iter=8; mean(abs(feat))=1448.2;
1
iter=9; mean(abs(feat))=1638.48;
1
iter=10; mean(abs(feat))=1589.6;
1
iter=11; mean(abs(feat))=1647.17;
1
iter=12; mean(abs(feat))=1769.25;
1
iter=13; mean(abs(feat))=1524.04;
1
iter=14; mean(abs(feat))=1660.6;
1
iter=15; mean(abs(feat))=1712.89;
1
iter=16; mean(abs(feat))=1856.9;
1
iter=17; mean(abs(feat))=2035.2;
1
iter=18; mean(abs(feat))=1855.03;
1
iter=19; mean(abs(feat))=2101.78;
1
iter=20; mean(abs(feat))=2024.25;
1
iter=21; mean(abs(feat))=2057.42;
1
iter=22; mean(abs(feat))=2390.24;
1
iter=23; mean(abs(feat))=2378.4;
1
iter=24; mean(abs(feat))=2129.09;
1
iter=25; mean(abs(feat))=2454.77;
1
iter=26; mean(abs(feat))=2369.3;
1
iter=27; mean(abs(feat))=2474.32;
1
iter=28; mean(abs(feat))=2632

KeyboardInterrupt: 

['net.conv4a.register_forward_hook(hook)']

In [33]:
feature_mask.shape

(1, 512, 4, 14, 14)